# Boost your own depth
This Colab Notebook helps you applying the global boosting method from https://github.com/compphoto/BoostingMonocularDepth 
on your own depth. It executes a single pass through the merging network, that we use to generate the **double estimation**.

Instructions:

1. Ensure using a GPU by setting "Runtime/change runtime type" to GPU
2. Run code section 1
3. Put your low-resolution base depth inside '/content/BoostYourOwnDepth/input/low-res'
4. (If you want to calculate the R20 resolution for your image, you can do the following:
        a.) Put the RGB image into '/content/BoostYourOwnDepth/input/rgb'
        b.) Run section 2 to get the optimal input resolution for a given image
        c.) Create a depth estimation from your network, with the resolution from b.) as input size
    )
5. Put your high-resolution depth inside '/content/BoostYourOwnDepth/input/high-res'
6. Run code section 3
    
The results will be generated in '/content/outputs'.


## Section 1
This section is cloning the repository and downloads the weights for the merging network:

In [ ]:
# Code section 1

# Clone repo
!git clone https://github.com/compphoto/BoostYourOwnDepth.git

# Downloading merge model weights
# !wget https://sfu.ca/~yagiz/CVPR21/latest_net_G.pth
!gdown https://drive.google.com/u/0/uc?id=1cU2y-kMbt0Sf00Ns4CN2oO9qPJ8BensP&export=download

!mkdir -p /content/BoostYourOwnDepth/pix2pix/checkpoints/mergemodel/
!mv /content/latest_net_G.pth /content/BoostYourOwnDepth/pix2pix/checkpoints/mergemodel/

!ls /content/BoostYourOwnDepth/pix2pix/checkpoints/mergemodel/


## Section 2
Run the next section to calculate the R20 resolution for an image of your choice. **IMPORTANT**: This will **NOT** do the actual depth estimation.

In [ ]:
# Code section 2
from BoostYourOwnDepth.utils import calculateprocessingres, read_image
import os

############################################################################
#### Set the image name that you want to optimize for, must be present in /content/BoostYourOwnDepth/input/rgb ######################
image_name = ''
############################################################################

image_path = os.path.join('/content/BoostYourOwnDepth/input/rgb',image_name)
img = read_image(image_path)

# Hyperparameters
whole_size_threshold = 3000  # R_max from the paper
r_threshold_value = 0.2 # Value x of R_x defined in the section 5 of the main paper.
scale_threshold = 3  # Allows up-scaling with a scale up to 3

##########################################################################
##### Fill in the receptive field size of your network here ##############
net_receptive_field_size = 
##########################################################################

# Find the best input resolution R-x. The resolution search described in section 5-double estimation of the main paper and section B of the
# supplementary material.
whole_image_optimal_size, _ = calculateprocessingres(img, net_receptive_field_size,r_threshold_value, scale_threshold, whole_size_threshold)

## Section 3
Put your low-res and high-res depth map in the corresponding folders in '/content/BoostYourOwnDepth/input/'.
Then, you can run the following section to generate the merged estimation. The section executes a single pass through our merging network to create a double estimation from your inputs. **IMPORTANT**: Please make sure that high-res and low-res files have the same name to avoid reading errors:

In [ ]:
# Code section 3
!python3 /content/BoostYourOwnDepth/boost_depth.py --data_dir /content/BoostYourOwnDepth/input/ --output_dir /content/BoostYourOwnDepth/output --checkpoints_dir /content/BoostYourOwnDepth/pix2pix/checkpoints --colorize_results

## Feel free to experiment!

E.g. edit the input depth maps, mix'n'match different depth estimators or use your own model.